<a href="https://colab.research.google.com/github/t1nh233/Pytorch-AI-Research-Roadmap/blob/main/Day_02_Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- STARTUP BLOCK ---
import os
import torch
import numpy as np

# 1. Kết nối Google Drive (Để lấy dữ liệu)
from google.colab import drive
drive.mount('/content/drive')

# 2. Tạo đường dẫn tắt (Để code cho gọn)
# Lưu ý: Sửa lại đúng tên thư mục bạn tạo ở Bước 2 phần trên
DATA_PATH = '/content/drive/MyDrive/Pytorch_AI_Research_Bootcamp/datasets'
MODEL_PATH = '/content/drive/MyDrive/Pytorch_AI_Research_Bootcamp/checkpoints'

# 3. Kiểm tra GPU
if torch.cuda.is_available():
    print(f"✅ Đã bật GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Cảnh báo: Bạn đang chạy bằng CPU. Hãy vào Runtime -> Change runtime type -> T4 GPU")

# 4. Tạo thư mục nếu chưa có (Tránh lỗi)
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

print("🚀 Môi trường đã sẵn sàng!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã bật GPU: Tesla T4
🚀 Môi trường đã sẵn sàng!


In [6]:
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = x**2 + 3*x + 1
z = y.sum()
print(y)
print(z)

tensor([11., 19.], grad_fn=<AddBackward0>)
tensor(30., grad_fn=<SumBackward0>)


In [7]:
#Goi backward

z.backward(retain_graph=True)
print(x.grad)
z.backward()
print(x.grad)

tensor([7., 9.])
tensor([14., 18.])


In [8]:
#Vay neu toi boc tat ca trong no_grad() thi sao

with torch.no_grad():
  k = x**2 + 3*x + 1

print(y.grad_fn)
print(k.requires_grad)
print(k.grad_fn)
print(y.grad_fn.next_functions)

False
None
((<AddBackward0 object at 0x7c386bd1cee0>, 0), (None, 0))


In [9]:

##Chuan bi du lieu

raw_diamond = torch.tensor([[1.5, 0.8], [0.5, 0.4], [2, 0.9]], requires_grad=False)
print(raw_diamond)

diamond_tabular = raw_diamond / torch.max(raw_diamond)
print(diamond_tabular)

diamond_image = torch.randn((3, 1, 4, 4), requires_grad=False)
print(diamond_image)

diamond_priceR = torch.tensor([1500.0, 500.0, 2000.0])
print(diamond_priceR)

diamond_priceRnorm = diamond_priceR / diamond_priceR.max()
print(diamond_priceRnorm)

print("Data prepared!!")

tensor([[1.5000, 0.8000],
        [0.5000, 0.4000],
        [2.0000, 0.9000]])
tensor([[0.7500, 0.4000],
        [0.2500, 0.2000],
        [1.0000, 0.4500]])
tensor([[[[-0.2541,  0.4529,  1.2718,  1.0151],
          [ 0.2275, -0.5807, -0.2573,  0.2834],
          [-0.1986, -0.3139,  0.0368,  0.9139],
          [ 0.0987, -0.3849, -1.0427,  0.7825]]],


        [[[-0.7687, -0.3188,  0.4002,  1.2794],
          [ 0.5446,  1.4101, -0.1606, -0.4163],
          [ 0.6195, -0.7006,  0.0750, -0.4892],
          [-0.3916,  0.0206,  0.4990,  1.6342]]],


        [[[ 0.3879,  0.5597,  1.1674, -0.7826],
          [ 0.4957,  0.3181,  1.7421, -0.4969],
          [ 1.3161, -0.2912, -1.3985,  0.1451],
          [-0.5638, -1.8148,  3.2028, -0.0343]]]])
tensor([1500.,  500., 2000.])
tensor([0.7500, 0.2500, 1.0000])
Data prepared!!


In [10]:
##Viet ham autograd thu cong

class MagicSquare(torch.autograd.Function):
  @staticmethod
  def forward(ctx, input):
    ctx.save_for_backward(input)
    return input**2

  @staticmethod
  def backward(ctx, grad_output):
    input, = ctx.saved_tensors
    grad_input = 2 * input * grad_output
    return grad_input

def magic_square(input):
  return MagicSquare.apply(input)

In [11]:
##Build model va kiem tra graph

w1 = torch.tensor([100.0, 100.0], requires_grad=True)
w2 = torch.tensor([50.0], requires_grad=True)

def forward_pass(input_tabular, input_image):
  diaimage_mean = input_image.mean(dim=(1, 2, 3))
  val_image = magic_square(diaimage_mean * w2)

  val_tabular = torch.matmul(input_tabular, w1)
  return val_tabular + val_image

diamond_pred = forward_pass(diamond_tabular, diamond_image)
print(f"Giá dự đoán: {diamond_pred}")
print(f"Grad Fn: {diamond_pred.grad_fn}")

Giá dự đoán: tensor([156.0559, 147.3188, 297.5814], grad_fn=<AddBackward0>)
Grad Fn: <AddBackward0 object at 0x7c386c468f70>


In [12]:
##Training loop

learning_rate = 0.01

print("Start training!!")
for epoch in range(100):
  diamond_predict = forward_pass(diamond_tabular, diamond_image)
  loss = (diamond_predict - diamond_priceRnorm).pow(2).mean()
  loss.backward()

  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    w1.grad.zero_()
    w2.grad.zero_()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss}")

print(f"Final Loss: {loss.item():.4f}")
print(f"w1 learned: {w1}")
print(f"w2 learned: {w2}")

Start training!!
Epoch: 0, Loss: 44569.88671875
Epoch: 10, Loss: 8867.8955078125
Epoch: 20, Loss: 6591.31640625
Epoch: 30, Loss: 5018.56787109375
Epoch: 40, Loss: 3826.915283203125
Epoch: 50, Loss: 2918.867919921875
Epoch: 60, Loss: 2226.586669921875
Epoch: 70, Loss: 1698.771484375
Epoch: 80, Loss: 1296.3455810546875
Epoch: 90, Loss: 989.5206909179688
Final Loss: 776.2282
w1 learned: tensor([10.7526, 55.6056], requires_grad=True)
w2 learned: tensor([0.0023], requires_grad=True)


In [13]:
##Kiem tra nang cao (detatch)

diamond_detached = diamond_predict.detach()
print(diamond_detached.requires_grad)
print(diamond_detached.grad_fn)

False
None


In [21]:
# w3 = torch.tensor([14.8, 23.3], requires_grad=True)
# wa, wb = w3.unbind()
# wa += 9.0
# w3.backward()
# print(w3.grad)


w3 = torch.tensor([14.8, 23.3], requires_grad=True)
wa, wb = w3.unbind()

wa.retain_grad()
wb.retain_grad()

losses = wa + 9.0
losses.retain_grad()
losses.backward()
print(w3.grad)
print(losses.grad)
print(wa.grad)
print(wb.grad)

tensor([1., 0.])
tensor(1.)
tensor(1.)
None
